# documentation 

generate and provide backend for services demonstrated in the documention for these notebooks



## convert notebooks to API


convert notebook to open api?


TODO: add check for readme.md in notebooks for description pages

TODO: finish export and convert other documentation to the open API format



#### the code

ROUTE[] = /swagger

ROUTE[] = /swagger/v1/swagger.json

TODO: console.log really good use of what attributes should be used for



In [ ]:
const path = require('path');
const pkg = require('../package.json');

// Imports
const { functionCache } = importer.import('cache rpc functions with llm descriptions');
const { cacheCells } = importer.import('cache notebook');
const { cellCache } = importer.import('cell cache')
const parseEnvironment = importer.import('parse environment');

// Path + ID helpers
const getRelativePath = filename => path.relative(path.resolve(__dirname, '..'), filename);
const removeExt = str => str.replace(/\.ipynb$/i, '');
const cleanPath = str => str.replace(/\s+/g, '').replace(/[^a-z0-9_/]/gi, '');
const convertToResourcePath = str => str.replace(/\//g, '.resources.').toLowerCase();
const getPathId = filename => convertToResourcePath(cleanPath(removeExt(getRelativePath(filename))));

// Path traversal
const getTail = (dotPath, ctx) => dotPath.split('.').reduce((acc, key) => acc[key] || (acc[key] = {}), ctx);

// Condition helpers
const hasExport = cache => !!(((cache || {}).exports || [])[0]);
const hasValidRoute = env => typeof env.ROUTE === 'string' || Array.isArray(env.ROUTE);
const hasQuestions = cell => Array.isArray(cell.questions) && !!cell.questions[0];
const shouldSkip = (cache, env, cell) => !(hasExport(cache) || hasValidRoute(env)) || !hasQuestions(cell);

// Env parsing
const getRouteStr = env => Array.isArray(env.ROUTE) ? env.ROUTE[0] : env.ROUTE;
const extractRoute = env => (getRouteStr(env) || '').replace(/^\/|\/$/g, '').split('/')[0];

// Function extraction
const extractFn = (cache, route) => ((cache || {}).exports || [])[0] || route;

// Parameters builder
const makeParam = key => ({
    type: 'string',
    location: 'query',
    required: true,
    description: ''
});
const buildParameters = cache => ((cache || {}).exports || []).slice(1).reduce((params, key) => {
    params[key] = makeParam(key); return params;
}, {});

// Tail method constructor
const addMethodToTail = (tail, fn, cell, cache, pathId) => {
    const desc = (cache || {}).summary || cell.markdown;
    const id = `${pkg.name}.${pathId}.${fn}`;
    const pathQ = `?function=${cell.questions[0]}`;
    tail.methods = tail.methods || {};
    tail.methods[fn] = { description: desc, scopes: ['Function', 'Available', 'Public'], id, httpMethod: 'GET', path: pathQ, parameters: buildParameters(cache) };
    return tail;
};

// Core builder
const buildRPCObject = (obj, cell) => {
    const cache = functionCache[path.join(path.dirname(cell.filename), cell.id)];
    const env = parseEnvironment(cell.markdown);
    if (shouldSkip(cache, env, cell)) return obj;
    const fn = extractFn(cache, extractRoute(env));
    const pathId = getPathId(cell.filename);
    const tail = getTail(pathId, obj);
    return Object.assign(tail, addMethodToTail(tail, fn, cell, cache, pathId)), obj;
};

// Final generator
const getCells = search => search
    ? (console.log('searching:', search), importer.interpret(search))
    : (console.log('searching all notebooks:'), cellCache.map(([_, cell]) => importer.lookupCell(cell, cacheCells)));

const getRpcSpecFromCells = (search) => {
    const then = Date.now();
    const cells = getCells(search);
    console.info('search took:', ((Date.now() - then) / 1000) + 's');

    const start = Date.now();
    const resources = cells.reduce(buildRPCObject, {});
    console.info('structuring took:', ((Date.now() - start) / 1000) + 's');

    return {
        title: 'Jupyter Ops',
        name: pkg.name,
        description: pkg.description,
        protocol: 'rest',
        baseUrl: process.env.FUNCTIONS_URL,
        resources,
        auth: {},
        schemas: {},
        parameters: {}
    };
};

module.exports = getRpcSpecFromCells;


#### test convert notebook api?


In [6]:
var importer = require('../Core');
var getRpcSpecFromCells = importer.import('convert notebook to open api');
var getRpcFromSpec = importer.import('get rpc from spec');
var getEnvironment = importer.import('get environment');
var groupPermissions = importer.import('test get permissions');
getEnvironment('STUDY_LOCAL')

if(typeof $$ !== 'undefined') {
    $$.async()
    var spec = getRpcSpecFromCells(['study sauce.ipynb', 'rpc.ipynb'])
//    var spec = getRpcSpecFromCells('study sauce.ipynb');
    console.log(spec)
    var juypter_ops = getRpcFromSpec(spec)
    console.log(juypter_ops)
//    juypter_ops.marketing.studysauce.copyStudy({email: 'megamindbrian@gmail.com'})
    juypter_ops.core.rpc.getPermissions({search: null})
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


importing convert notebook to open api - 1 cell - documentation.ipynb[0]
compiling /Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[0] aka convert notebook to open api with module,importer,exports,__dirname,__filename,$$
Error: Line 2: Unexpected token :
    at ErrorHandler.constructError (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:5012:22)
    at ErrorHandler.createError (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:5028:27)
    at Parser.unexpectedTokenError (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:1985:39)
    at Parser.throwUnexpectedToken (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:1995:21)
    at Parser.consumeSemicolon (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:2297:23)
    at Parser.parseExpressionStatement (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:3649:15)
    at Parser.parseStatement (/Users/briancu

Error: connect ECONNREFUSED 127.0.0.1:8010

### convert google discovery to documentation?

https://developers.google.com/apis-explorer/#s/discovery/v1/discovery.apis.getRest?api=discovery&version=v1&_h=2&

https://www.googleapis.com/discovery/v1/apis/discovery/v1/rest

https://developers.google.com/apis-explorer/#p/discovery/v1/discovery.apis.list?_h=1&




discover well-known apis?

TODO: figure out how OpenAPI, Swagger RAML do this, Google's is obvious

https://tools.ietf.org/html/rfc5785


## Export docs


### export documentation?

ROUTE[] = /docs

ROUTE[] = /docs/*

ROOT = true

DEFAULT = true


In [ ]:
import fs from 'fs'
import path from 'path'
import mustache from 'mustache'

const { listInProject } = importer.import('list project files')
const {
  renderSearchPage,
  renderHomepage,
  renderCategoryPage,
} = importer.import('render template pass through')
const renderNotebook = importer.import('render notebook mustache')

const NOTEBOOK_PATH = path.dirname(__dirname)

const categorizeNotebook = filepath => {
  const [category = 'uncategorized'] = path.relative(NOTEBOOK_PATH, filepath).split(path.sep)
  return { category, namespace: path.basename(filepath, '.ipynb'), relative: filepath }
}


const writePageToDisk = (filename, html) => {
  const outputPath = path.join(__dirname, '..', 'docs', filename)
  fs.mkdirSync(path.dirname(outputPath), { recursive: true })
  fs.writeFileSync(outputPath, html)
}

export default async function exportAll(req, res, next) {
  const isServe = typeof req != 'undefined'
  const docs = req && req.route.path === '*' ? '' : ('/' + req.route.path.split('/')[1])
  const url = decodeURIComponent(req ? req.path || '' : '').replace(docs, '').replace(/^\//, '')
  const TEMPLATES = importer.interpret('docs mustache template').code

  const notebooks = listInProject(NOTEBOOK_PATH, '{,*,*/*,*/*/*,*/*/*/*}/*.ipynb')
    .map(file => Object.assign({ file }, categorizeNotebook(file)))

  const seenCategories = notebooks.map(n => n.category).filter((a, i, arr) => arr.indexOf(a) === i)

  let allPages = []
  let possiblyValidCategory = url.replace('/index.html', '').split('/').pop()
  if (isServe ? seenCategories.includes(possiblyValidCategory) : true) {
    const matching = notebooks.filter(n => n.category === possiblyValidCategory)
    allPages.push(renderCategoryPage(seenCategories, possiblyValidCategory, matching, TEMPLATES, isServe ? url : true))
  }
  if (isServe ? !seenCategories.includes(possiblyValidCategory) : true) {
    const allNotebooks = notebooks.flatMap(({ file, category, namespace }) => {
      const pages = []
      const matching = notebooks.filter(n => n.category === category)

      const { pages: notebookPages, cells } = renderNotebook(file, category, namespace, { notebooks: matching, isServe, exactRoute: isServe ? url : true, TEMPLATES })
      return { pages: pages.concat(notebookPages), cells, category, namespace }
    })

    allPages = allPages.concat(allNotebooks.flatMap(n => n.pages))
  }

  if (isServe ? new RegExp('^/(search(\\.html)?)?$').test(url) : true)
    allPages.push(renderSearchPage(seenCategories, allNotebooks, TEMPLATES))

  if (isServe ? (url === '' || new RegExp('^/*(index\\.html)?$').test(url)) : true)
    allPages.push(renderHomepage(seenCategories, TEMPLATES))

  console.log(possiblyValidCategory, url, docs, allPages)

  for (const page of allPages) {
    const html = mustache.render(importer.interpret('javadoc template').code, page)
    if (isServe && url.replace(/\/*index.html/, '') === page.filename.replace(/\/*index.html/, '')) return res.send(html)
    if (!isServe) writePageToDisk(page.filename, html)
  }

  next()
}



### mustache templates

docs mustache template?


In [ ]:
{{#cellPage}}
<h1>{{{TITLE}}}</h1>
{{#QUESTION}}
<h2>Run example</h2>
<pre language="bash"><code>npm run import -- "{{QUESTION}}"</code></pre>
{{/QUESTION}}
{{{SUMMARY}}}
<div class="code-block">
  <pre class="{{LANGUAGE}}">
    <code>{{CODE}}</code>
  </pre>
</div>
{{{AMAZING}}}
{{{HIGHLIGHTER}}}
{{/cellPage}}
{{#amazing}}
<div class="gold">
  <h2>What the code could have been:</h2>
  {{{CODE}}}
  {{{DESCRIPTION}}}
</div>
{{/amazing}}
{{#searchPage}}
<h1>Search</h1>
{{#CATEGORIES}}
<h2>{{TITLE}}</h2>
{{#NOTEBOOKS}}
<h3>{{TITLE}}</h3>
{{#CELLS}}
<a href="./{{CATEGORY}}/{{NAMESPACE}}/{{NAME}}.html">{{TITLE}}</a>
{{SUMMARY}}
{{/CELLS}}
{{/NOTEBOOKS}}
{{/CATEGORIES}}
{{/searchPage}}
{{#homePage}}
<h1>Categories</h1>
{{#CATEGORIES}}
<a href="./{{TITLE}}/index.html">{{TITLE}}</a><br /><br />
{{/CATEGORIES}}
{{/homePage}}
{{#categoryPage}}
<h1>{{CATEGORY}}</h1>
<br />
{{#NOTEBOOKS}}
<a href="./{{NAMESPACE}}/index.html">{{TITLE}}</a><br /><br />
{{/NOTEBOOKS}}
{{/categoryPage}}
{{#notebookPage}}
<h1>{{NAMESPACE}}</h1>
{{#CELLS}}
<a href="./{{NAME}}.html">{{TITLE}}</a><br />
{{SUMMARY}}
<br />
{{/CELLS}}
{{/notebookPage}}


### render code

code fix render template?


In [ ]:
const { Remarkable } = require('remarkable');
const md = new Remarkable({ html: true, xhtmlOut: true, breaks: true });

function renderCode(description, language) {
  const codeBlockRegex = /```(\w*)\n([\s\S]*?)(\n```|$)/g
  const parts = []
  let lastIndex = 0
  let match

  while ((match = codeBlockRegex.exec(description)) !== null) {
    const [fullMatch, lang = '', code] = match
    const index = match.index

    // Push preceding markdown
    if (index > lastIndex) {
      const markdown = description.slice(lastIndex, index)
      parts.push(md.render(markdown))
    }

    // Push rendered code block
    parts.push(`<div class="code-block">${md.render(`\`\`\`${lang}\n${code}\n\`\`\``)}</div>`)
    lastIndex = index + fullMatch.length
  }

  // Push any remaining markdown after the last code block
  if (lastIndex < description.length) {
    parts.push(md.render(description.slice(lastIndex)))
  }

  // If no code blocks at all, treat the whole thing as a code snippet
  if (parts.length === 0) {
    return `<div class="code-block">${md.render(`\`\`\`${language}\n${description}\n\`\`\``)}</div>`
  }

  return parts.join('\n')
}

module.exports = renderCode



### template pass through

render template pass through?


In [ ]:
const mustache = require('mustache')
const { Remarkable } = require('remarkable');
const md = new Remarkable({ html: true, xhtmlOut: true, breaks: true });

const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const { safeurl } = importer.import('domain cache tools')

const renderMarkdown = (description) => md.render(description)

const renderAmazingBlock = (DESCRIPTION, CODE, LANGUAGE, TEMPLATES) =>
  mustache.render(TEMPLATES, { amazing: { CODE, LANGUAGE, DESCRIPTION } })

const renderCell = (index, cell, cache, SUMMARY, AMAZING, PLUGIN, TEMPLATES) =>
  mustache.render(TEMPLATES, {
    cellPage: {
      TITLE: cell.questions[0] || 'Cell ' + index,
      QUESTION: cell.question || '',
      CODE: cell.code,
      LANGUAGE: cell.language,
      HIGHLIGHTER: mustache.render(importer.interpret('highlight js').code, { LANGUAGE: cell.language, PLUGIN }),
      AMAZING,
      SUMMARY,
    }
  })

const renderNotebookIndex = ({ notebooks, notebookPath, category, namespace, cells }, TEMPLATES, exactRoute) => ({
  TITLE: `${namespace}.ipynb`,
  filename: `${category}/${namespace}/index.html`,
  OUTPUT: mustache.render(TEMPLATES, {
    notebookPage: {
      CATEGORY: category, NAMESPACE: namespace, CELLS: cells.map((cell, i) => ({
        TITLE: getCellName(cell, i),
        NAME: (exactRoute ? '' : (namespace + '/'))
          + safeurl(getCellName(cell, i).toLocaleLowerCase()),
        SUMMARY: (functionCache[`${notebookPath}[${i}]`] || {}).summary
      }))
    }
  }),
  SEARCH: `<a href="${exactRoute ? '../' : ''}../search.html">Search</a>`,
  PREV: (n => n ? `<a href="./${exactRoute ? '../' : ''}${n.namespace}">${n.namespace}</a>` : '')
    (notebooks[notebooks.map(n => n.namespace).indexOf(namespace) - 1]),
  NEXT: (n => n ? `<a href="./${exactRoute ? '../' : ''}${n.namespace}">${n.namespace}</a>` : '')
    (notebooks[notebooks.map(n => n.namespace).indexOf(namespace) + 1]),
  CLASSES: `<h3><a href="./${exactRoute ? '../' : ''}index.html">${category}</a></h3>\n\n<br /><br />`
    + notebooks.map(n => `<a href="./${exactRoute ? '../' : ''}${n.namespace}">${n.namespace}</a>`).join('\n\n<br /><br />'),
  PARENT: `<strong><a href="./${exactRoute ? '../' : ''}index.html">${category}</a></strong>`,
  isNotebookIndex: true
})

const renderCategoryPage = (categories, category, notebooks, TEMPLATES, exactRoute) => ({
  TITLE: `${category}`,
  filename: `${category}/index.html`,
  OUTPUT: mustache.render(TEMPLATES, {
    categoryPage: {
      CATEGORY: category + ': Namespaces',
      NOTEBOOKS: notebooks.map(n => ({ NAMESPACE: (exactRoute ? '' : (category + '/')) + n.namespace, TITLE: n.namespace }))
    }
  }),
  SEARCH: `<a href="${exactRoute ? '../' : './'}search.html">Search</a>`,
  CLASSES: `<h3><a href="../index.html">Home</a></h3>\n\n<br /><br />`
    + categories.map(c => `<a href="${exactRoute ? '../' : './'}${c}/index.html">${c}</a>`).join('\n\n<br /><br />'),
  PREV: ((i) => categories[i - 1] ? `<a href="./${categories[i - 1]}">${categories[i - 1]}</a>` : void 0)(categories.indexOf(category)),
  NEXT: ((i) => categories[i + 1] ? `<a href="./${categories[i + 1]}">${categories[i + 1]}</a>` : void 0)(categories.indexOf(category)),
  PARENT: `<strong><a href="../index.html">Home</a></strong>`,
  isCategory: true
})

const renderHomepage = (categories, TEMPLATES) => ({
  TITLE: 'Jupyter Docs',
  filename: 'index.html',
  OUTPUT: mustache.render(TEMPLATES, {
    homePage: {
      CATEGORIES: categories.map(TITLE => ({ TITLE })) 
    }
  }),
  isIntro: true
})

const renderSearchPage = (categories, notebooks, TEMPLATES) => ({
  TITLE: 'Search',
  filename: 'search.html',
  OUTPUT: mustache.render(TEMPLATES, {
    searchPage: {
      categories: categories.map(title => ({
        TITLE: title,
        NOTEBOOKS: notebooks
          .filter(n => n.category === title)
          .map(n => ({ TITLE: n.namespace, CELLS: n.cells }))
      }))
    }
  }),
  isSearch: true
})

const getCellName = (cell, i) => !cell ? void 0 : ((cell || {}).questions || [])[0] ? cell.questions[0] : 'Cell ' + i

module.exports = {
  renderSearchPage,
  renderHomepage,
  renderCategoryPage,
  renderNotebookIndex,
  renderCell,
  renderAmazingBlock,
  renderMarkdown,
  getCellName,
}


### render entire notebook

render notebook mustache?


In [ ]:
const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const { safeurl } = importer.import('domain cache tools')
const {
  renderNotebookIndex,
  renderCell,
  renderAmazingBlock,
  renderMarkdown,
  getCellName,
} = importer.import('render template pass through')
const renderCode = importer.import('code fix render template')

function buildCellPage({ i, cells, category, namespace, notebookPath, TEMPLATES }) {
  const cell = cells[i]
  const cellname = getCellName(cell, i)
  const filename = `${category}/${namespace}/${safeurl(cellname.toLowerCase())}.html`
  const prev = getCellName(cells[i - 1], i - 1)
  const next = getCellName(cells[i + 1], i + 1)
  const cache = functionCache[`${notebookPath}[${i}]`] || {}

  const CLASSES = `<h3><a href="./index.html">${namespace}</a></h3><br /><br />` +
    cells.map((c, j) => {
      const name = getCellName(c, j)
      return `<a href="./${safeurl(name.toLowerCase())}.html">${name}</a>`
    }).join('\n<br /><br />')

  return {
    TITLE: `${namespace}.ipynb: ${cellname}`,
    filename,
    OUTPUT: renderCell(
      i,
      cell,
      cache,
      renderMarkdown(cache.summary || cell.markdown),
      renderAmazingBlock(
        renderMarkdown(cache.description),
        renderCode(cache.amazing || '', cell.language),
        cell.language,
        TEMPLATES
      ),
      '../../../mergehtml.js',
      TEMPLATES
    ),
    category,
    namespace,
    index: i,
    PREV: prev ? `<a href="./${safeurl(prev.toLowerCase())}.html">${prev}</a>` : undefined,
    NEXT: next ? `<a href="./${safeurl(next.toLowerCase())}.html">${next}</a>` : undefined,
    PARENT: `<strong><a href="./index.html">${namespace}</a></strong>`,
    SEARCH: `<a href="../../../search.html">Search</a>`,
    CLASSES
  }
}

function renderNotebook(notebookPath, category, namespace, { notebooks, isServe, exactRoute, TEMPLATES }) {
  const renderIndex = exactRoute ? new RegExp(`${category}/${namespace}(/index\\.html)?`).test(exactRoute) : true
  const renderCells = exactRoute ? new RegExp(`${category}/${namespace}/.*\\.html$`).test(exactRoute) : true
  const cells = (renderIndex || renderCells) ? importer.interpret(notebookPath) : []
  const pages = []

  if (renderIndex) {
    const isExactIndex = exactRoute === true ? true : exactRoute.endsWith('/index.html')
    pages.push(renderNotebookIndex({ notebooks, notebookPath, category, namespace, cells }, TEMPLATES, isExactIndex))
  }

  if (renderCells) {
    for (let i = 0; i < cells.length; i++) {
      const filename = `${category}/${namespace}/${safeurl(getCellName(cells[i], i).toLowerCase())}.html`
      if (isServe && !exactRoute.endsWith(filename)) continue
      pages.push(buildCellPage({ i, cells, category, namespace, notebookPath, TEMPLATES }))
      if (isServe) break
    }
  }

  return { pages, cells, category, namespace }
}

module.exports = renderNotebook



highlight js?

so someday these can be like mustache templates or w/e


ROUTE[] = /mergehtml.js

ROUTE[] = /docs/mergehtml.js

STATIC = path.join(__dirname, '..', 'docs', 'mergehtml.js')



In [ ]:
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/atom-one-dark.min.css">


<script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/highlight.min.js"></script>

<!-- and it's easy to individually load additional languages -->
<script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/languages/{{LANGUAGE}}.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/languages/bash.min.js"></script>
<script src="{{PLUGIN}}"></script>

<script>
let codeCells = document.querySelectorAll('.code-block code')
let startLine = 0
for(let i = 0; i < codeCells.length; i++) {
  let lineCode = document.createElement('code')
  lineCode.classList.add('lines')
  let lines = codeCells[i].innerHTML.replace(/[\s\n\r]{0,2}$/, '').split('\n')
  lineCode.innerHTML = lines.map((line, i) => '<p id="' + (startLine + i + 1) + '"><a class="number" href="#' + (startLine + i + 1) + '">' + (i + 1) + '</a>' + (line.trim().length == 0 ? '&nbsp;' : line) + '</p>').join('')
  codeCells[i].parentElement.insertAdjacentElement('afterend', lineCode)
  codeCells[i].addEventListener('dblclick', (function (html) {
    navigator.clipboard.writeText(html)
  }).bind(null, codeCells[i].innerText))
  startLine += lines.length
}
hljs.addPlugin(mergeHTMLPlugin);
hljs.highlightAll();

</script>




javadoc template?


In [ ]:
<!DOCTYPE html>
<html>

<head>
  <meta charset='utf-8'>
  <meta http-equiv='X-UA-Compatible' content='IE=edge'>
  <title>{{TITLE}}</title>
  <meta name='viewport' content='width=device-width, initial-scale=1'>
  <link rel="stylesheet" href="/docs-style.css" />
</head>

<body>
  {{#CLASSES}}
  <nav>
    {{{CLASSES}}}
  </nav>
  {{/CLASSES}}
  {{#PARENT}}
  <header>
    {{{PARENT}}} | {{#PREV}}{{{PREV}}} |{{/PREV}} {{#NEXT}}{{{NEXT}}} |{{/NEXT}} {{{SEARCH}}}
  </header>
  {{/PARENT}}

  {{{OUTPUT}}}
</body>

</html>


documentation stylesheet?

ROUTE = /docs-style.css


In [ ]:

* {
  margin: 0;
  padding: 0;
  box-sizing: border-box;
}

html {
  padding: 0;
  margin: 0;
}

nav {
  position: fixed;
  overflow: auto;
  top: 0;
  left: 0;
  right: auto;
  bottom: 0;
  width: 200px;
}

header {
  background-color: #EEE;
  padding: 10px;
}

body {
  padding: 0 0 0 200px;
  margin: 0;
}

.code-block {
  position: relative;
}

.gold pre code,
.gold pre code span,
.gold code pre,
.gold code pre span {
  color: gold;
}

.lines {
  position: absolute;
  top: 0;
  left: 0;
  /* z-index: -1; */
  pointer-events: none;
  color: transparent;
  padding-right: 20px;
  white-space: pre-wrap;
  font-size: 16px;
  padding-left: 5em;
}

.lines a:not(.number) {
  color: transparent;
}

.lines p {
  background-color: transparent;
  position: relative;
  padding-top: 100px;
  margin-top: -100px;
}

@keyframes flash {
  0% {
    background-color: transparent;
    opacity: 1;
  }

  22% {
    background-color: yellow;
  }

  77% {
    background-color: yellow;
  }

  100% {
    background-color: transparent;
  }
}

.lines p:active .number,
.lines p:focus .number,
.lines p:target .number {
  animation-name: flash;
  animation-duration: 900ms;
  animation-iteration-count: 1;
  animation-timing-function: ease-in-out;
  transition: background-color 1s ease-in-out;
}

.lines .bookmark {
  top: -200px;
  position: absolute;
}

.lines .number {
  color: #fff;
  width: 60px;
  text-align: right;
  display: inline-block;
  text-decoration: none;
  pointer-events: all;
  position: absolute;
  padding-right: 1em;
  margin-left:-5em;
}

.lines .number:active,
.lines .number:focus,
.lines .number:target {
  color: lightblue;
}

code {
  max-height: 999999px;
  text-size-adjust: none;
}

pre {
  white-space: pre-wrap;
  font-size: 16px;
  max-height: 999999px;
  text-size-adjust: none;
}

body pre code.hljs {
  padding: 0;
  padding-left: 5em;
}

@media screen and (max-width: 600px) {
  body {
    padding-left: 0;
  }

  nav {
    display: none;
  }
}
